# LA Crime: Database models

#### Load Python tools and Jupyter config

In [34]:
import os
import json
import requests
import pandas as pd
import jupyter_black
from sodapy import Socrata

In [35]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [36]:
today = pd.Timestamp("today").strftime("%Y%m%d")
today_epoch = int(pd.Timestamp.today().timestamp())

---

## Read

In [37]:
app_token = os.environ.get("SOCRATA_APP_TOKEN")

In [44]:
def fetch_data_with_pagination_with_progress(base_url, limit=1000, max_iterations=10):
    offset = 0
    all_records = []
    current_iteration = 0

    print("Fetching data...")
    while current_iteration < max_iterations:
        query_url = f"{base_url}&$limit={limit}&$offset={offset}"
        response = requests.get(query_url)
        data = response.json()

        if not data:
            print("No more data to fetch.")
            break  # Stop if there are no more records

        all_records.extend(data)
        offset += limit
        current_iteration += 1

        print(f"Progress: {current_iteration}/{max_iterations} iterations completed.")

    return pd.DataFrame(all_records)

base_url = "https://data.lacity.org/resource/2nrs-mtv8.json?$order=date_rptd DESC"
df = fetch_data_with_pagination_with_progress(base_url)

In [47]:
df.head()

,dr_no,date_rptd,date_occ,time_occ,area,area_name,rpt_dist_no,part_1_2,crm_cd,crm_cd_desc,vict_age,premis_cd,premis_desc,status,status_desc,crm_cd_1,location,cross_street,lat,lon,mocodes,vict_sex,vict_descent,crm_cd_2,weapon_used_cd,weapon_desc,crm_cd_3
0,240507457,2024-04-01T00:00:00.000,2024-03-31T00:00:00.000,2300,05,Harbor,0557,1,510,VEHICLE - STOLEN,0,101,STREET,IC,Invest Cont,510,4TH ST,CABRILLO AV,33.7406,-118.2946,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,241209315,2024-04-01T00:00:00.000,2024-03-31T00:00:00.000,2300,12,77th Street,1252,1,510,VEHICLE - STOLEN,0,101,STREET,IC,Invest Cont,510,2700 W 76TH ST,NaN,33.9709,-118.3232,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,242007939,2024-04-01T00:00:00.000,2024-03-29T00:00:00.000,1000,20,Olympic,2027,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)",51,122,"VEHICLE, PASSENGER/TRUCK",IC,Invest Cont,740,500 S KENMORE AV,NaN,34.0654,-118.2966,1814 2000 0329 2021,F,K,NaN,NaN,NaN,NaN
3,240109818,2024-04-01T00:00:00.000,2024-04-01T00:00:00.000,1100,01,Central,0192,1,510,VEHICLE - STOLEN,0,101,STREET,IC,Invest Cont,510,W PICO BL,MARGO ST,34.0386,-118.2638,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,241707752,2024-04-01T00:00:00.000,2024-03-01T00:00:00.000,0800,17,Devonshire,1708,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),0,101,STREET,IC,Invest Cont,420,12300 MCLENNAN AV,NaN,34.2932,-118.501,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.to_parquet('../data/raw/updated_incidents_table.parquet', index=False)